# Hand-in Group
| Name            | Matriculation number |  TUM ID |
|-----------------|--------|----------------------|
| Mohammed Elbushnaq | 03786474 | go56cuh |
| Marina Aoki     | 03773384 | ge94naz |
| Sama Elbaroudy  | 03768259 | ge83muj |
| Sarah Berbuir   | 03767700 | ge83cak |

# Preparation

## Install Dependencies

In [ ]:
!pip install datasets
!pip install tokenizers
!pip install transformers
!pip install stanza
# -- Initialize Stanza --
import stanza
stanza.download('en')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 12.3 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.1/990.1 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.8/433.8 kB 16.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none

INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Downloading default packages for language: en (English) ...


INFO:stanza:Downloaded file to /root/stanza_resources/en/default.zip
INFO:stanza:Finished downloading models and saved to /root/stanza_resources


## Download Datasets
Note: In case the following script fails because the medical transcriptions dataset is no longer available it can be downloaded from https://www.kaggle.com/tboyle10/medicaltranscriptions .

In [ ]:
# -- OPEN-I dataset --
!mkdir -p open_i
!wget -q -N -P open_i https://openi.nlm.nih.gov/imgs/collections/NLMCXR_reports.tgz
!tar zxf open_i/NLMCXR_reports.tgz -C open_i
import glob
num_files = len(glob.glob("open_i/ecgen-radiology/*.xml"))
print(f'Downloaded {num_files} files from the OpenI dataset into "open_i/ecgen-radiology"')

# -- Transcriptions Dataset --
!mkdir -p medical_transcriptions
!gdown --id 1E0hm3r9bwK8cujyIcOjp_y-ZEPt1HBjn -O medical_transcriptions/medical_transcriptions.zip
!unzip -o medical_transcriptions/medical_transcriptions.zip -d medical_transcriptions
print(f'Downloaded medical transriptions dataset')




Downloaded 3955 files from the OpenI dataset into "open_i/ecgen-radiology"
/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:132: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1E0hm3r9bwK8cujyIcOjp_y-ZEPt1HBjn
To: /content/medical_transcriptions/medical_transcriptions.zip
100% 5.08M/5.08M [00:00<00:00, 55.4MB/s]
Archive:  medical_transcriptions/medical_transcriptions.zip
  inflating: medical_transcriptions/mtsamples.csv  
Downloaded medical transriptions dataset


# Task 1: Pre-Processing and Vocabulary Construction (not graded!)

## Note
- Task 1 is not graded, i.e. it is not mandatory to solve this task. It insteads serves soleley to show how pre-processing and tokenization can be done.
- Note that tasks 2 and 3 are mandatory!

## Goals
- Understand tools for text pre-processing
- Apply text normalization and sentence splitting to a medical corpus
- Train a tokenizer on a medical corpus

## Tools
- [Huggingface Datasets](https://huggingface.co/docs/datasets/): Library for efficient loading, saving and processing of text datasets
  - Note: In many cases using pandas is sufficient for text datasets (as they often fit into memory), but Hugginface Datasets provides some usful features like batched mapping of samples.
- [Huggingface Tokenizers](https://huggingface.co/docs/tokenizers/python/latest/): Library for text normalization and tokenization
- [Stanza](https://stanfordnlp.github.io/stanza/): Text processing toolkit from the Stanford NLP group
   - usefull for sentence splitting

## Dataset
OpenI [[Website](https://openi.nlm.nih.gov/faq#collection)] [[Download](https://openi.nlm.nih.gov/imgs/collections/NLMCXR_reports.tgz)]
- Contains reports and scans
- We only use the reports from this dataset

## Step 1 - Text Normalization

In this step the goal is to normalize text from the corpus.

TODO: Implement a function for normalizing a single text sample (e.g. a report from). This function will later be applied to all sample of the corpus.
You can decide which text normalization you find appropriate and implement one or some of them. Typical examples are:
- Unicode normalization
- Stripping accents
- Lowercasing
- Removing control characters
- Normalizing whitespace characters (i.e. replacing all whitespaces like tabs, ... by the default whitespace) and removing redundant whitespaces
- Removing or normalizing special characters

Note: you may implement each normalizazion using regex/string operations or you can use normalizers from https://huggingface.co/docs/tokenizers/python/latest/api/reference.html#module-tokenizers.normalizers

In [ ]:
from tokenizers.normalizers import *

def normalize_text(text: str) -> str:
  normaliser = BertNormalizer(strip_accents=True)
  return normaliser.normalize_str(text)





In [ ]:
# apply normalization to some example text
# you can play around with different example texts to understand the effects of each normalizer
example_text = 'Schöner Tag'

print(normalize_text(example_text))

schoner tag


## Step 2 - Sentence Splitting

In this step the goal is to split each section into sentences.

TODO: Implement a function for splitting a single report section (given as a single string) into sentences (returned as a list of strings, one string for each sentence). This function will later be applied to all samples of the corpus.
For sentence splitting the stanza tokenizer should be used as it provides a more robust solution compared to splitting on pre-defined characters (like ".") .


In [ ]:
import stanza
from typing import List

stanza_tokenizer = stanza.Pipeline('en', processors='tokenize', use_gpu=True)

def split_sentences(text: str) -> List[str]:
  # See https://stanfordnlp.github.io/stanza/tokenize.html#tokenization-and-sentence-segmentation
  return [token.text for sentence in stanza_tokenizer(text).sentences for token in sentence.tokens]


INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |
| mwt       | combined |

INFO:stanza:Using device: cuda
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: mwt
INFO:stanza:Done loading processors!


In [ ]:
# apply sentence splitting on some example text
example_text = "This is the first sentence. Let's try something else, e.g. this. Another sentence?"

print(split_sentences(example_text))

['This', 'is', 'the', 'first', 'sentence', '.', "Let's", 'try', 'something', 'else', ',', 'e.g.', 'this', '.', 'Another', 'sentence', '?']


## Step 3 - Apply text normalization and sentence splitting to the Open-I dataset (nothing TODO here)
We now apply text normalization and sentences splitting to the Open-I dataset.
Each sample of the Open-I dataset contains a report with (besides others) a "findings" and "impression" section.

We make use of the huggingface datasets library for efficiently applying text normalization and sentence splitting to both sections of each sample of the Open-I dataset.
This step is already implemented but utilizes the functions implemented in Step 1 and 2.


1. Load the Open-I dataset as a huggingface Dataset

In [ ]:
from typing import List
import glob
from tqdm import tqdm
import xml.etree.ElementTree as ET
from datasets import Dataset

def load_open_i_dataset():
  # Load from xml-files
  uIds = []
  findings_sections = []
  impression_sections = []
  for file in tqdm(glob.glob("open_i/ecgen-radiology/*.xml")):
    tree = ET.parse(file)

    uId = tree.find("./uId").get('id')
    uIds.append(uId)

    finding = tree.find(".//AbstractText[@Label='FINDINGS']").text
    findings_sections.append(finding if finding is not None else '')

    imp = tree.find(".//AbstractText[@Label='IMPRESSION']").text
    impression_sections.append(imp if imp is not None else '')

  assert len(uIds) > 0, 'No data found. Download the data first.'
  return Dataset.from_dict({
      'uId': uIds,
      'findings': findings_sections,
      'impression': impression_sections
  })

data = load_open_i_dataset()
print(data) # Overview over the dataset
print(data[0])  # The first sample of the dataset
# Save to be loaded later
data.save_to_disk('open_i_raw')

100%|██████████| 3955/3955 [00:00<00:00, 5031.07it/s]


Dataset({
    features: ['uId', 'findings', 'impression'],
    num_rows: 3955
})
{'uId': 'CXR3713', 'findings': '', 'impression': 'Heart size is at upper limits normal. Mediastinal silhouette otherwise and pulmonary vascularity is within normal limits. Left chest wall changes suggest prior thoracotomy. No pleural effusion or pneumothorax. No focal airspace consolidation.'}


Saving the dataset (0/1 shards):   0%|          | 0/3955 [00:00<?, ? examples/s]

2. Apply normalize_text to the findings and impression sections using the map function of datasets.Dataset

In [ ]:
from datasets import load_from_disk, Dataset

data: Dataset = load_from_disk('open_i_raw')
def normalize_text_samples(sample: dict) -> dict:
  return {
      'findings_normalized': normalize_text(sample['findings']),
      'impression_normalized': normalize_text(sample['impression'])
  }
data = data.map(normalize_text_samples)

print(data) # Overview over the dataset
print(data[0])  # The first sample of the dataset

Map:   0%|          | 0/3955 [00:00<?, ? examples/s]

Dataset({
    features: ['uId', 'findings', 'impression', 'findings_normalized', 'impression_normalized'],
    num_rows: 3955
})
{'uId': 'CXR3713', 'findings': '', 'impression': 'Heart size is at upper limits normal. Mediastinal silhouette otherwise and pulmonary vascularity is within normal limits. Left chest wall changes suggest prior thoracotomy. No pleural effusion or pneumothorax. No focal airspace consolidation.', 'findings_normalized': '', 'impression_normalized': 'heart size is at upper limits normal. mediastinal silhouette otherwise and pulmonary vascularity is within normal limits. left chest wall changes suggest prior thoracotomy. no pleural effusion or pneumothorax. no focal airspace consolidation.'}


3. Apply split_sentences to the findings and impression sections using the map function of datasets.Dataset

In [ ]:
def split_sample_sentences(sample: dict) -> dict:
  return {
      'findings_sentences': split_sentences(sample['findings']),
      'impression_sentences': split_sentences(sample['impression'])
  }

split_dataset = data.map(split_sample_sentences)

print(split_dataset)
print(split_dataset[0])
split_dataset.save_to_disk('open_i_split')





Map:   0%|          | 0/3955 [00:00<?, ? examples/s]

Dataset({
    features: ['uId', 'findings', 'impression', 'findings_normalized', 'impression_normalized', 'findings_sentences', 'impression_sentences'],
    num_rows: 3955
})
{'uId': 'CXR3713', 'findings': '', 'impression': 'Heart size is at upper limits normal. Mediastinal silhouette otherwise and pulmonary vascularity is within normal limits. Left chest wall changes suggest prior thoracotomy. No pleural effusion or pneumothorax. No focal airspace consolidation.', 'findings_normalized': '', 'impression_normalized': 'heart size is at upper limits normal. mediastinal silhouette otherwise and pulmonary vascularity is within normal limits. left chest wall changes suggest prior thoracotomy. no pleural effusion or pneumothorax. no focal airspace consolidation.', 'findings_sentences': [], 'impression_sentences': ['Heart', 'size', 'is', 'at', 'upper', 'limits', 'normal', '.', 'Mediastinal', 'silhouette', 'otherwise', 'and', 'pulmonary', 'vascularity', 'is', 'within', 'normal', 'limits', '.', 

Saving the dataset (0/1 shards):   0%|          | 0/3955 [00:00<?, ? examples/s]

## Step 4 - Vocabulary Creation: Train a tokenizer
In this step the goal is to learn a vocabulary (i.e. tokenizer) from the Open-I sentences. We therefore extract a list of all sentences (from findings and impression section) in the dataset. This list is then used for learning the vocabulary.

TODO: Extract all sentences and store them in all_sentences.

In [ ]:
from datasets import load_from_disk, Dataset
from typing import List
from tqdm import tqdm

split_dataset: Dataset = load_from_disk('open_i_split')
print(split_dataset)

all_sentences: List[str] = [sentence for sentences in split_dataset["findings_sentences"][1:] for sentence in sentences]+[sentence for sentences in split_dataset["impression_sentences"][1:] for sentence in sentences]

print(f'\nTotal number of sentences: {len(all_sentences)}')
print(f'Total number of words (by whitespaces): {sum(len(sent.split()) for sent in all_sentences)}')
num_unique_words = len(set(word for sent in all_sentences for word in sent.split()))
print(f'Total number of different words: {num_unique_words}')


Dataset({
    features: ['uId', 'findings', 'impression', 'findings_normalized', 'impression_normalized', 'findings_sentences', 'impression_sentences'],
    num_rows: 3955
})

Total number of sentences: 178954
Total number of words (by whitespaces): 178954
Total number of different words: 2653


We no define the initial alphabet (optional, by default the alphabet is derived from the characters present in the dataset) and the size of the vocabulary we want to create.

In [ ]:
import string

# All ascii characters (lowercase only) + digits
initial_alphabet = list(string.ascii_lowercase) + list(string.digits)
print(f'Initial alphabet size: {len(initial_alphabet)}')

vocab_size = 50000

Initial alphabet size: 36


Now we use the huggingface tokenizer library to learn a WordPiece tokenizer. This is already implemented but you can try with different arguments or also try other tokenizers, e.g. BPE.

In [ ]:

from tokenizers import Tokenizer, models, normalizers, pre_tokenizers, decoders, trainers, processors

special_tokens = ["<PAD>", "<UNK>", "<BOS>", "<EOS>"]
tokenizer = Tokenizer(models.WordPiece(unk_token='<UNK>'))
# we use a lowercase vocabulary
tokenizer.normalizer = normalizers.BertNormalizer(lowercase=True)
tokenizer.pre_tokenizer = pre_tokenizers.BertPreTokenizer()
tokenizer.post_processor = processors.TemplateProcessing(
    single="<BOS> $0 <EOS>",
    pair="<BOS> $A <EOS> <BOS> $B <EOS>",
    special_tokens=[("<BOS>", 2), ("<EOS>", 3)],
)
tokenizer.decoder = decoders.WordPiece()

trainer = trainers.WordPieceTrainer(
    vocab_size=vocab_size,
    initial_alphabet=initial_alphabet,
    special_tokens=special_tokens,
)

# now train it
tokenizer.train_from_iterator(all_sentences, trainer=trainer)

print(f'Vocab size: {tokenizer.get_vocab_size()}')
print('Vocab: \n', tokenizer.get_vocab())

tokenizer.save('tokenizer.json', pretty=True)

Vocab size: 4309
Vocab: 
 {'fluoroscop': 2736, 'note': 1479, '##verses': 3732, 'proced': 2014, 'xray': 4206, 'ng': 2597, '##raclavicular': 3811, 'suboptimal': 3315, 'comminuted': 3235, 'superimposes': 3900, 'part': 1782, '##idging': 2670, '##lusion': 3488, '##ed': 148, 'ven': 3437, '##flow': 2637, '##val': 505, 'shoul': 1116, 'resid': 1167, 'gastrostomy': 3224, 'pat': 462, 'unrem': 312, 'hodgk': 4170, '##sal': 2934, 'communication': 4123, '##t': 61, 'uniformly': 4221, 'cta': 3847, '##dal': 2643, 'pattern': 1456, '##hange': 3677, 'results': 2107, '##ication': 3610, 'hyperdensity': 3808, 'unfolded': 1907, '##asm': 1766, '##ize': 2438, '##diaphragm': 581, 'pseudofissure': 4307, '##act': 290, 'ne': 1808, 'accepted': 4212, '##cin': 2638, 'laterally': 3075, 'focused': 4005, 'tortuosity': 826, '##im': 196, 'develop': 1120, 'knees': 3186, 'motion': 2066, 'distrib': 3641, 'caval': 2045, 'obscure': 1530, 'noncalcified': 1855, 'abnormality': 246, '##clavicular': 3512, '##ific': 417, 'pur': 3411, 

### Inspection of the created vocabulary
You can now take a look at the created vocabulary by opening the file "tokenizer.json".

You can also try around tokenizing some samples:

In [ ]:
example_section = split_dataset['findings_sentences']
example_sentence = example_section[1]

print(example_sentence)

encoded = tokenizer.encode(" ".join(example_sentence))
print(f'Encoded tokens: {encoded.tokens}')
print(f'Encoded token ids: {encoded.ids}')
print(f'Decoded again: {tokenizer.decode(encoded.ids)}')
print(f'Decoded (keep special tokens): {tokenizer.decode(encoded.ids, skip_special_tokens=False)}')

['The', 'heart', 'size', 'is', 'at', 'the', 'upper', 'limits', 'of', 'normal', '.', 'The', 'mediastinum', 'is', 'unremarkable', '.', 'There', 'is', 'no', 'pleural', 'effusion', ',', 'pneumothorax', ',', 'or', 'focal', 'airspace', 'disease', '.', 'Mild', 'chronic', 'degenerative', 'changes', 'are', 'present', 'within', 'the', 'thoracic', 'spine', '..']
Encoded tokens: ['<BOS>', 'the', 'heart', 'size', 'is', 'at', 'the', 'upper', 'limits', 'of', 'normal', '.', 'the', 'mediastinum', 'is', 'unremarkable', '.', 'there', 'is', 'no', 'pleural', 'effusion', ',', 'pneumothorax', ',', 'or', 'focal', 'airspace', 'disease', '.', 'mild', 'chronic', 'degenerative', 'changes', 'are', 'present', 'within', 'the', 'thoracic', 'spine', '.', '.', '<EOS>']
Encoded token ids: [2, 98, 172, 181, 113, 529, 98, 367, 213, 140, 121, 11, 98, 319, 113, 313, 11, 169, 113, 95, 161, 151, 9, 168, 9, 160, 193, 260, 204, 11, 284, 549, 356, 292, 112, 485, 206, 98, 318, 300, 11, 11, 3]
Decoded again: the heart size is at t

# Task 2: Training a Text Classifier (graded!)
## Goals
- Understand how pre-trained language models can be utilized for downstream tasks

## Tools
- [Huggingface Transformers](https://huggingface.co/transformers/): Library for pre-trained transformer-based language models)

## Dataset
Medical Transcriptions [[Kaggle](https://www.kaggle.com/tboyle10/medicaltranscriptions)]

## Step 1 - Load the Medical Transctions Dataset (Nothing TODO here)
We first load the Medical Transcriptions dataset as a huggingface dataset. The dataset is split into train and test set, so the returned type is a datasets.DatasetDict, which acts like a dictionary of datasets.Dataset but provides utility functions, e.g. for mapping all datasets of the dict using the map method.
This step is already implemented.

In [ ]:
from typing import List, Tuple
import pandas as pd
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split

def load_transciptions_dataset() -> Tuple[DatasetDict, List[str]]:
    df = pd.read_csv('medical_transcriptions/mtsamples.csv')
    df = df.drop(['Unnamed: 0'],axis=1,)

    counts = df['medical_specialty'].value_counts()
    print(f'Original counts:\n{counts}')
    dropped_specialities = [k for k, v in counts.items() if v < 100]
    for dropped_speciality in dropped_specialities:
      df = df[df['medical_specialty'] != dropped_speciality]
    df.dropna(inplace=True)
    counts = df['medical_specialty'].value_counts()
    print(f'Counts after removing small specialities:\n{counts}')

    df['medical_specialty'] = df['medical_specialty'].astype('category')
    class_names = df['medical_specialty'].cat.categories.tolist()
    print(f'Class names : {class_names}')
    df['medical_specialty'] = df['medical_specialty'].cat.codes

    train, test = train_test_split(df, stratify=df['medical_specialty'], test_size=0.25)
    dataset = DatasetDict({'train': Dataset.from_pandas(train), 'test': Dataset.from_pandas(test)})
    dataset = dataset.remove_columns(['__index_level_0__'])

    return dataset, class_names

In [ ]:
dataset, class_names = load_transciptions_dataset()

Original counts:
medical_specialty
 Surgery                          1103
 Consult - History and Phy.        516
 Cardiovascular / Pulmonary        372
 Orthopedic                        355
 Radiology                         273
 General Medicine                  259
 Gastroenterology                  230
 Neurology                         223
 SOAP / Chart / Progress Notes     166
 Obstetrics / Gynecology           160
 Urology                           158
 Discharge Summary                 108
 ENT - Otolaryngology               98
 Neurosurgery                       94
 Hematology - Oncology              90
 Ophthalmology                      83
 Nephrology                         81
 Emergency Room Reports             75
 Pediatrics - Neonatal              70
 Pain Management                    62
 Psychiatry / Psychology            53
 Office Notes                       51
 Podiatry                           47
 Dermatology                        29
 Dentistry                   

In [ ]:
print('Training samples: ', len(dataset['train']['transcription']))
print('Test samples: ', len(dataset['test']['transcription']))
print('Columns: ', dataset['train'].features)

Training samples:  2315
Test samples:  772
Columns:  {'description': Value(dtype='string', id=None), 'medical_specialty': Value(dtype='int8', id=None), 'sample_name': Value(dtype='string', id=None), 'transcription': Value(dtype='string', id=None), 'keywords': Value(dtype='string', id=None)}


## Step 2 - Compute sentence embeddings using pre-trained language model
Our goal is to use a pre-trained language model (from the huggingface transformers library) to encode sentences into sentence representations. Therefore, each sentence is tokenized and passed to the language model. The resulting contextualized token representations (i.e. the outputs of the last hidden layer of the language model) are then globally pooled to get sentence-level representations. As we use a pre-trained model, no training is involved in this step.

### Specify pre-trained language model
TODO: Decide which pre-trained language model you want to use and specify its name here.
You can search for models at the huggingface model hub: https://huggingface.co/models . You can either use standard models, e.g. BERT, or use biomedcial models.

When you decided for a model, copy the name of the model from the URL (removing only https://huggingface.co/) and insert it as model_name here.
For more reference on how to use the tokenizer have a look at https://huggingface.co/docs/transformers/preprocessing.

In [ ]:
model_name = "bert-base-cased"

In [ ]:
# Load the tokenizer (nothing TODO)
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
print(tokenizer)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

BertTokenizerFast(name_or_path='bert-base-cased', vocab_size=28996, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}


In [ ]:
# Load the model (nothing TODO) and inspect it
from transformers import AutoModel
model = AutoModel.from_pretrained(model_name)
print(model)

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(28996, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
  

### Experiment with tokenizer and language model (nothing to implement here)

In [ ]:
# Tokenize some example text and inspect the results to understand the outputs of the tokenizer
sample_text = dataset['train'][:2]['transcription']  # batch of 2 samples of text
print('example text: ', sample_text)
toknized = tokenizer(sample_text, truncation=True, max_length=512)
print(toknized.keys())
print('input ids: ', toknized['input_ids'])
print('attention mask: ', toknized['attention_mask'])
print('length: ', len(toknized['input_ids'][0]))



example text:  ["PREOPERATIVE DIAGNOSES:,  Left calcaneal valgus split.,POSTOPERATIVE DIAGNOSES:,  Left calcaneal valgus split.,PROCEDURES:  ,1.  Left calcaneal lengthening osteotomy with allograft.,2.  Partial plantar fasciotomy.,3.  Posterior subtalar and tibiotalar capsulotomy.,4.  Short leg cast placed.,ANESTHESIA: , Surgery performed under general anesthesia.,TOURNIQUET TIME: , 69 minutes.,The patient in local anesthetic of 20 mL of 0.25% Marcaine plain.,COMPLICATIONS: , No intraoperative complications.,DRAINS:  ,None.,SPECIMENS: , None.,HISTORY AND PHYSICAL: , The patient is a 13-year-old female who had previous bilateral feet correction at 1 year of age.  Since that time, the patient has developed significant calcaneal valgus deformity with significant pain.  Radiographs confirmed collapse of the spinal arch, as well as valgus position of the foot.  Given the patient's symptoms, surgery is recommended for calcaneal osteotomy and Achilles lengthening.  Risks and benefits of surge

In [ ]:
# Feed some example data to understand the outputs of the language model

# pad the batch and convert it to Pytorch
# return_tensors='pt' => return the tokenized values as PyTorch tensors instead of lists
x = tokenizer.pad(toknized, return_tensors='pt')
print(x.keys())
print('input ids: ', x['input_ids'])
print('attention mask: ', x['attention_mask'])
print('shape: ', x['input_ids'].shape)

# Encode the tokenized and padded input
results = model(**x)
print('output shape: ', results.last_hidden_state.shape)  # (batch_size x num_tokens x d_hidden)

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])
input ids:  tensor([[  101, 11629,  2036,  ...,   119,  1109,   102],
        [  101,   141,  6258,  ...,  1131,  1125,   102]])
attention mask:  tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1]])
shape:  torch.Size([2, 512])
output shape:  torch.Size([2, 512, 768])


### Tokenize the dataset
We now tokenize the whole dataset using the batched map-method.

TODO: Implement the tokenize_batch function using the tokenizer. Note: do not pad the data yet but truncate it to a max length of 512.

In [ ]:
def tokenize_batch(text_batch: List[str]):
  return tokenizer(text_batch, max_length=512, truncation=True)

In [ ]:
# Apply tokenize_batch to dataset (nothing TODO here)
tokenized_dataset = dataset.map(lambda examples: tokenize_batch(examples["transcription"]), batched=True)
tokenized_dataset = tokenized_dataset.remove_columns(['sample_name', 'transcription', 'description', 'keywords']).rename_column('medical_specialty', 'labels')
print('Columns after tokenization', list(tokenized_dataset['train'].features.keys()))

Map:   0%|          | 0/2315 [00:00<?, ? examples/s]

Map:   0%|          | 0/772 [00:00<?, ? examples/s]

Columns after tokenization ['labels', 'input_ids', 'token_type_ids', 'attention_mask']


In [ ]:
tokenized_dataset["train"]

Dataset({
    features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 2315
})

### Token Pooling
Define Pooling functions to compute sentence embeddings from token outputs.

TODO: implement three possible pooling functions
- CLS: use the output of the [CLS] token only and ignore the other tokens.
- max/avg: globally max/avg pool over all token outputs, but make sure to ignore padding tokens.

In [ ]:
import torch

def CLS_pool(hidden_state: torch.FloatTensor, attention_mask: torch.BoolTensor):
  """
  Returns only the hidden state of the [CLS] token
  :param hidden_state: (N x M x d_hidden) where N is the batch size and M is the number of tokens
  :param attention_mask: (N x M), True for "real" tokens, False for padding tokens.
  :return (N x d_hidden)
  """
  return hidden_state[:, 0, :]

def max_pool(hidden_state: torch.FloatTensor, attention_mask: torch.BoolTensor):
  """
  Globally pools the hidden states over all tokens using max pooling.
  :param hidden_state: (N x M x d_hidden) where N is the batch size and M is the number of tokens
  :param attention_mask: (N x M), True for "real" tokens, False for padding tokens.
  :return (N x d_hidden)
  """
  return torch.max(hidden_state * attention_mask.unsqueeze(-1), dim=1)

def avg_pool(hidden_state: torch.FloatTensor, attention_mask: torch.BoolTensor):
  """
  Globally pools the hidden states over all tokens using average pooling.
  :param hidden_state: (N x M x d_hidden) where N is the batch size and M is the number of tokens
  :param attention_mask: (N x M), True for "real" tokens, False for padding tokens.
  :return (N x d_hidden)
  """
  return hidden_state.sum(axis=1) / attention_mask.sum(axis=-1).unsqueeze(-1)

### Sentence Embedder
The sentence embedder takes a tokenized input, uses the language model to compute token representations (i.e. the last hidden_state of the language model) and then uses a pooling function to compute a single sentence representation.

TODO: implement the sentence embedder forward method.

In [ ]:
from torch import nn
from typing import Dict

class SentenceEmbedder(nn.Module):
  def __init__(self, model, pool):
    super().__init__()
    self.model = model  # this is a huggingface language model from AutoModel.from_pretrained
    self.pool = pool  # this is one of the three defined pooling functions (CLS, max, avg)

  def forward(self, x: Dict[str, torch.Tensor]) -> torch.Tensor:
    """
    :param x: dict containing the following elements:
      - input_ids: torch.Tensor of the token_type ids with shape (N x M)
      - attention_mask: torch.Tensor containing the attention mask of shape (N x M)
    :return sentence_embedding for each sample of shape (N x d_hidden)
    """
    output = model(**x)
    hidden_state = output.last_hidden_state
    return self.pool(hidden_state, x["attention_mask"])


TODO: now decide which pooling function you want to use

In [ ]:
# TODO: try different pooling functions (nothing more to implement here)
# (also change the dataset_name so you can later easily switch between pooling functions without the need to recompute the embeddings)
device = 'cuda:0'
dataset_name = 'encoded_transciptions_CLS'
pool = CLS_pool

sentence_embedder = SentenceEmbedder(model, pool).to(device=device)



 Now run the sentence embedder (nothing TODO here).

 The resulting dataset will then contain a sentence_embedding column.

In [ ]:
def embed_sentence(batch):
  model_input = {'input_ids': batch['input_ids'], 'attention_mask': batch['attention_mask']}
  model_input = tokenizer.pad(model_input, return_tensors='pt').to(device=device)

  with torch.no_grad():
    sentence_embeddings = sentence_embedder(model_input)
  return {'sentence_embedding': sentence_embeddings.detach().cpu().numpy()}

encoded_dataset = tokenized_dataset.map(embed_sentence, batched=True, batch_size=64)
encoded_dataset.save_to_disk(dataset_name)

Map:   0%|          | 0/2315 [00:00<?, ? examples/s]

Map:   0%|          | 0/772 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/2315 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/772 [00:00<?, ? examples/s]

## Step 5 - Train Classification Model on Sentence Embeddings
Now we have a single sentence representation vector for each sentence. We now learn a simple classifier based on a MLP (i.e. a 2-layer fully-connected neural network). We first project each sentence embedding from d_hidden to d_mlp, apply ReLU (or another non-linearity) and then project the resulting vector into num_classes where we then apply the cross entropy loss. The classification head will then be learned based on the training data. This is the only learned component in our model.

TODO: implement the MLP head

In [ ]:
from torch import nn
import datasets
from datasets import load_from_disk
from torch.utils.data import DataLoader
import torch
from tqdm import tqdm

class ClassificationHead(nn.Module):
  def __init__(self, d_hidden: int, d_mlp: int, num_classes: int):
    super().__init__()
    self.mlp_head = nn.Sequential(
        nn.Linear(d_hidden, d_mlp),
        nn.ReLU(),
        nn.Linear(d_mlp, num_classes)
    )

    self.loss = nn.CrossEntropyLoss()

  def forward(self, x, y_true):
    """
    :param x: sentence embeddings (N x d_hidden)
    :param y_true: target classes (multiclass) (N)
    """
    logits = self.mlp_head(x)

    loss = self.loss(logits, y_true)
    y_pred = logits.argmax(dim=1)

    return y_pred, loss



Now train the classification head on the sentence embeddings.
The training is already implemented.

TODO: specify and try different hyperparameters

In [ ]:
# TODO: try different hyperparameters
device = 'cuda:0'
dataset_name = 'encoded_transciptions_CLS'
num_epochs = 50
lr = 1e-3
weight_decay = 1e-4
d_hidden = 768  # must match the language model hidden output
d_mlp = 1024
batch_size = 64

In [ ]:
# Now run the training...

print('num classes: ', len(class_names))
encoded_dataset = load_from_disk(dataset_name)
# make sure PyTorch Tensors are returned
encoded_dataset.set_format('pt')

# remove all columns except sentence_embedding and labels
columns_to_remove = set(encoded_dataset['train'].column_names) - {'sentence_embedding', 'labels'}
encoded_dataset = encoded_dataset.remove_columns(columns_to_remove)

train_data_loader = DataLoader(encoded_dataset['train'], batch_size=batch_size, shuffle=True)
test_data_loader = DataLoader(encoded_dataset['test'], batch_size=batch_size, shuffle=False)

classification_model = ClassificationHead(d_hidden=d_hidden,
                                          d_mlp=d_mlp,
                                          num_classes=len(class_names))
classification_model = classification_model.to(device=device)
optimizer = torch.optim.Adam(classification_model.parameters(), lr=lr, weight_decay=weight_decay)

classification_model.train()
print('Training...')
for epoch in range(num_epochs):
  train_loss = []
  for train_batch in tqdm(train_data_loader):
    x = train_batch['sentence_embedding'].to(device=device)
    y_true = train_batch['labels'].to(device=device)

    y_pred, loss = classification_model(x, y_true)
    train_loss.append(loss.item())

    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
  print('train loss: ', torch.mean(torch.tensor(train_loss)).item())

classification_model.eval()
print('Testing...')
f1_metric = datasets.load_metric("f1")
with torch.no_grad():
  test_loss = []
  for test_batch in tqdm(test_data_loader):
    x = test_batch['sentence_embedding'].to(device=device)
    y_true = test_batch['labels'].to(device=device)

    y_pred, loss = classification_model(x, y_true)

    f1_metric.add_batch(predictions=y_pred, references=y_true)
    test_loss.append(loss)
print('\ntest loss: ', torch.mean(torch.tensor(test_loss)).item())
print('F1 (Macro): ', f1_metric.compute(average="macro"))

num classes:  12
Training...


100%|██████████| 37/37 [00:01<00:00, 35.00it/s]


train loss:  2.0011377334594727


100%|██████████| 37/37 [00:00<00:00, 41.02it/s]


train loss:  1.7248567342758179


...

train loss:  0.878571093082428


100%|██████████| 37/37 [00:00<00:00, 308.52it/s]


train loss:  0.8660518527030945
Testing...


/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for f1 contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.1/metrics/f1/f1.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
100%|██████████| 13/13 [00:00<00:00, 340.84it/s]


test loss:  1.7874451875686646
F1 (Macro):  {'f1': 0.1965033771816821}


# Task 3: Generating text from a pre-trained decoder LM (graded!)
## Goals
- Understand how inference (text generation) works on decoder languages models

## Tools
- [Huggingface Transformers](https://huggingface.co/transformers/): Library for pre-trained transformer-based language models)

## Model
- Architecture: https://huggingface.co/docs/transformers/model_doc/gpt2#transformers.GPT2LMHeadModel
- Model weights: https://huggingface.co/healx/gpt-2-pubmed-medium

## Notes and Tips
- You can read this blog post to learn more about generation strategies: https://huggingface.co/blog/how-to-generate
- We are going to implement greedy-search.
- Please do not use the "generate" method of the pre-trained model (as there generation is already implemented)
- You can however take a look at their implementation of greedy-search: https://github.com/huggingface/transformers/blob/04ab5605fbb4ef207b10bf2772d88c53fc242e83/src/transformers/generation/utils.py#L2080


## Step 1: Implement generate method
Implement the generate method that takes a language model and its tokenizer together with a list of text prefixes and outputs a list of generated sentences (one for each prefix). The prefixes should be "autocompleted" by the model.
Use the greedy-search method and process all prefixes as a single batch.

TODO: finish the implementation of the generate method by filling in the missing lines.

In [ ]:
import torch
from typing import List

def generate(model, tokenizer, prefix: List[str], max_predicted: int) -> List[str]:
  """
  :param model: PreTrainedModel.
  :param tokenizer: PreTrainedTokenizer (https://huggingface.co/docs/transformers/main/main_classes/tokenizer#transformers.PreTrainedTokenizer)
  :prefix: Batch of prefixes to be autocompleted by the language model
  """
  if tokenizer.pad_token_id is None:
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.pad_token_id = tokenizer.eos_token_id
  pad_token_id = tokenizer.pad_token_id
  bos_token_id = tokenizer.bos_token_id
  eos_token_id = tokenizer.eos_token_id
  device = model.device

  # 1. Tokenize the prefixes and prepare them for input into the language model
  #    Notes
  #    - add the start token (BOS) but not the end token (EOS)
  #    - left! pad them to the maximum length in the batch, i.e. paddding is left to the "real" tokens
  #    - inputs_ids and attention_mask should be computes, and should already be stacked in the batch dim
  #    -> this can all be done by calling the tokenizer! See: https://huggingface.co/docs/transformers/main/en/internal/tokenization_utils#transformers.PreTrainedTokenizerBase.__call__
  tokenizer.padding_side = 'left'
  # TODO: call the tokenizer to prepare the inputs (a dictionary)
  inputs = tokenizer(prefix,  # add arguments here
                     padding=True,
                     truncation=True,
                     return_tensors='pt')
  inputs = inputs.to(device=device)
  input_ids = inputs['input_ids']
  attention_mask = inputs['attention_mask']

  # TODO: Compute position ids based on the attention mask
  # Note: the position ids are the indices of the positions, starting with zero and increasing for each non-padding token
  # The positions ids for padding tokens can take any value
  position_ids: torch.LongTensor = torch.cumsum(attention_mask, dim=1)

  # Initialize some variables (nothing TODO here)
  N = position_ids.shape[0]
  past_key_values = None
  unfinished_sequences = torch.ones(N, dtype=torch.bool, device=device)
  predicted_input_ids = input_ids.clone()

  while True:
    # 2. Predict the next token logits by passing the previous inputs into the language model
    #    Note: already computed steps can be given by past_key_values, other steps are given as input_ids
    outputs = model(
        input_ids=input_ids,
        attention_mask=attention_mask,
        position_ids=position_ids,
        past_key_values=past_key_values,
        return_dict=True
    )
    # (N x |V|)
    next_token_logits: torch.LongTensor = outputs.logits[:, -1, :]

    # 3. Select the next predicted tokens by greedy search
    # (N)
    next_tokens: torch.LongTensor = torch.argmax(next_token_logits, dim=1)  # TODO

    # 4. For all already finished sentences, replace the next_tokens by <PAD>
    next_tokens = torch.where(unfinished_sequences, next_tokens, torch.tensor(pad_token_id, device=device))  # TODO

    # 5. Check which sentences are already finished by checking whether they contain <END>
    # Which sentences have been finished with the predicted token (next_tokens)
    # (N)
    newly_finished = next_tokens == eos_token_id  # TODO
    # Which sentences, therefore, remain unfinished
    # (N)
    unfinished_sequences = unfinished_sequences & ~newly_finished  # TODO

    # 6. Concatenate the next input to predicted_input_ids, attention_mask
    # (N x M_new) where M_new is one longer than the previous M of predicted_input_ids
    predicted_input_ids = torch.cat((predicted_input_ids, next_tokens.unsqueeze(dim=1)), dim=1)  # TODO
    # (N x M_new)
    attention_mask = torch.cat((attention_mask, torch.ones((N, 1), dtype=torch.long, device=device)), dim=1)  # TODO: extend attention mask by ones for next step
    # Nothing TODO here
    position_ids = position_ids.amax(dim=1, keepdim=True) + 1
    input_ids = next_tokens.unsqueeze(dim=1)
    past_key_values = outputs.past_key_values

    # 7. Check if finished (nothing TODO here)
    if predicted_input_ids.shape[1] >= max_predicted or unfinished_sequences.max() == 0:
      break

  # 8. Convert back to text (nothing TODO here)
  generated_sentences: List[str] = tokenizer.batch_decode(predicted_input_ids, skip_special_tokens=True)
  return generated_sentences




## Step 2: Load a decoder language model (nothing TODO here)

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

decoder_model_name = "gpt2-medium"

decoder_tokenizer = AutoTokenizer.from_pretrained(decoder_model_name)
decoder_model = AutoModelForCausalLM.from_pretrained(decoder_model_name)
decoder_model = decoder_model.to(device="cuda:0")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/718 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.52G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

## Step 3: Apply the generate method to some example text (nothing TODO here)

In [ ]:
# You can play around with different texts here
prefix_1 = "This sentences is about"
prefix_2 = "Can you complete this sentence?"
prefixes = [prefix_1, prefix_2]

generated_sentences = generate(decoder_model, decoder_tokenizer, prefixes, max_predicted=50)

for i, (prefix, sent) in enumerate(zip(prefixes, generated_sentences)):
  print(f'Completed sentence {i}: "{prefix}"')
  print("-------------------------------------------------------\n")
  print(sent)
  print("\n=======================================================\n\n")

Completed sentence 0: "This sentences is about"
-------------------------------------------------------

This sentences is about about about about about about about about about about about about about about about about about about about about about about about about about about about about about about about about about about about about about about about about about about about about about



Completed sentence 1: "Can you complete this sentence?"
-------------------------------------------------------

Can you complete this sentence????????"""""""""""""""""""""""""""""""""""""





As you can see, the results tend to be quite repetetive.
This is why greedy search is typically not used in practice.
Common alternatives include beam search and sampling from the distribution of next tokens. While this is not part of the mandatory exercise, you can try to implement other generation methods as part of a bonus task and see how it can improve the results.

## Extra Task (not graded): Implement a beam search generate function and apply it
Re-implement the generate method but instead of doing greedy search, use beam search instead.

Note that this is a completely optional and non-graded task and will not be discussed in the soutions!

In [ ]:
def beam_search_generate(model, tokenizer, prefix: List[str], max_predicted: int, num_beams: int = 5) -> List[str]:
  pass  # TODO